Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [1]:
import numpy as np 
import pandas as pd 
from pandas.io.json import json_normalize
import json 
import requests
!conda install -c conda-forge wikipedia --yes 
from bs4 import BeautifulSoup

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
html_parsed_result = BeautifulSoup(website_url, 'html.parser') #transform the text to html

toronto_info_table = html_parsed_result.find('table', class_ = 'wikitable')
toronto_rows = toronto_info_table.find_all('tr')

# extract the info ('Postcode', 'Borough', 'Neighbourhood') from the table
toronto_info = []
for row in toronto_rows:
    info = row.text.split('\n')[1:-1] # remove empty str (first and last items)
    toronto_info.append(info)
    
toronto_info[0:10]

[['Postcode', 'Borough', 'Neighbourhood'],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned']]

In [3]:
#creating a Toronto Neighborhoods dataframe
toronto_df = pd.DataFrame(toronto_info[1:], columns=toronto_info[0])
toronto_df = toronto_df.rename(columns = {'Postcode':'Postal Code', 'Neighbourhood':'Neighborhood'})
toronto_df.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [5]:
#Cleaning the data
# skip not assigned boroughs:
toronto_df = toronto_df.drop(toronto_df[(toronto_df.Borough == "Not assigned")].index)
# give "Not assigned" Neighborhoods same name as Borough:
toronto_df.Neighborhood.replace("Not assigned", toronto_df.Borough, inplace=True)

In [6]:
# More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
toronto_df = toronto_df.groupby(['Postal Code','Borough'])['Neighborhood'].apply(', '.join).reset_index()
toronto_df.head(20)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [7]:
#in the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
toronto_df.shape

(103, 3)